In [4]:
# coding=utf-8
"""
LightGBM + SHAP 全流程分析示例（含兜底，静音版 + 保留原始项目顺序）
- 修复多分类 SHAP 计算（支持 list / 3D ndarray）
- 所有文件保存增加 PermissionError 兜底（自动改名 *_new.xxx）
"""

import os
import warnings
import logging

warnings.filterwarnings("ignore")
logging.getLogger("matplotlib.font_manager").setLevel(logging.ERROR)

# ====== 依赖检查 ======
required_libraries = [
    ("numpy", "np"),
    ("pandas", "pd"),
    ("sklearn", "sklearn"),
    ("lightgbm", "lgb"),
    ("matplotlib.pyplot", "plt"),
]
missing_libraries = []
for lib_name, _ in required_libraries:
    try:
        __import__(lib_name)
    except ImportError:
        missing_libraries.append(lib_name)
if missing_libraries:
    raise ImportError(f"缺少必要的库，请安装：{', '.join(missing_libraries)}")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
import lightgbm as lgb
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = ["SimHei", "Microsoft YaHei", "Arial"]
plt.rcParams["axes.unicode_minus"] = False

# ====== SHAP 导入 ======
try:
    import shap

    shap_available = True
except Exception as e:
    shap_available = False
    print(f"警告：SHAP 导入失败，将使用 LightGBM 兜底。原因：{e}")

# ====== 通用安全保存函数（处理 PermissionError） ======
def _make_alt_path(path: str) -> str:
    base, ext = os.path.splitext(path)
    if not ext:
        ext = ".txt"
    return f"{base}_new{ext}"

def safe_to_csv(df: pd.DataFrame, path: str, **kwargs):
    try:
        df.to_csv(path, **kwargs)
    except PermissionError:
        alt_path = _make_alt_path(path)
        print(f"[警告] 无法写入 {path}（权限不足或被占用），改写入 {alt_path}")
        df.to_csv(alt_path, **kwargs)

def safe_write_text(path: str, text: str, mode: str = "w", encoding: str = "utf-8"):
    try:
        with open(path, mode, encoding=encoding) as f:
            f.write(text)
    except PermissionError:
        alt_path = _make_alt_path(path)
        print(f"[警告] 无法写入 {path}（权限不足或被占用），改写入 {alt_path}")
        with open(alt_path, mode, encoding=encoding) as f:
            f.write(text)

def safe_save_fig(path: str):
    try:
        plt.savefig(path)
    except PermissionError:
        alt_path = _make_alt_path(path)
        print(f"[警告] 无法写入图片 {path}（权限不足或被占用），改写入 {alt_path}")
        plt.savefig(alt_path)

# ====== 配置 ======
DATA_PATH = r"E:\科研\深度学习框架--推荐系统\user_item_purchase_matrix_full_latest_with_capacity.csv"
RANDOM_SEED = 42
TEST_RATIO = 0.2
VALID_RATIO = 0.1
N_ESTIMATORS = 2000
EARLY_STOPPING_ROUNDS = 100
TOP_N = 50

OUTPUT_DIR = os.path.join(os.path.dirname(DATA_PATH), "lgbm_shap_outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ====== 读取数据 ======
print("Loading data from:", DATA_PATH)
df = pd.read_csv(DATA_PATH, low_memory=False)

user_id_col = df.columns[0]
target_col = df.columns[-1]
feature_cols = df.columns[1:-1]

print(f"UserID column: '{user_id_col}'")
print(f"Target column: '{target_col}'")
print(f"Number of features: {len(feature_cols)}")

X_df = df[feature_cols].copy()
y_raw = df[target_col]

# 特征预处理：统一为 0/1 的 uint8
for c in feature_cols:
    X_df[c] = pd.to_numeric(X_df[c], errors="coerce").fillna(0)
    X_df[c] = X_df[c].clip(0, 1).astype(np.uint8)

# 目标编码
if y_raw.dtype.kind in "bifc":
    y = y_raw.astype(int).values
else:
    y, _ = pd.factorize(y_raw)

classes = np.unique(y)
print(f"Unique classes in target: {classes}")

# ====== 划分训练 / 验证 / 测试 ======
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_df, y, test_size=TEST_RATIO, stratify=y, random_state=RANDOM_SEED
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full,
    y_train_full,
    test_size=VALID_RATIO,
    stratify=y_train_full,
    random_state=RANDOM_SEED,
)

# 类别权重
class_weights = compute_class_weight("balanced", classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
sample_weights = np.array([class_weight_dict[cls] for cls in y_train])

print(
    f"Training samples: {X_train.shape[0]}, "
    f"Validation samples: {X_valid.shape[0]}, "
    f"Test samples: {X_test.shape[0]}"
)

# ====== LightGBM 模型训练 ======
is_multiclass = len(np.unique(y)) > 2
params = {
    "objective": "multiclass" if is_multiclass else "binary",
    "random_state": RANDOM_SEED,
    "n_estimators": N_ESTIMATORS,
    "learning_rate": 0.05,
    "num_leaves": 63,
    "verbose": -1,
    "n_jobs": -1,
    "metric": "multi_logloss" if is_multiclass else "binary_logloss",
}
if is_multiclass:
    params["num_class"] = len(np.unique(y))

model = lgb.LGBMClassifier(**params)

print("Training LightGBM model (quiet)...")
model.fit(
    X_train,
    y_train,
    sample_weight=sample_weights,
    eval_set=[(X_valid, y_valid)],
    eval_metric="multi_logloss" if is_multiclass else "logloss",
    callbacks=[
        lgb.early_stopping(EARLY_STOPPING_ROUNDS, verbose=False),
        lgb.log_evaluation(0),
    ],
)

# ====== 测试集评估 ======
print("Evaluating on test data...")
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)
report_path = os.path.join(OUTPUT_DIR, "classification_report.txt")
safe_write_text(report_path, report, encoding="utf-8")

# ====== 特征重要性（Gain） ======
print("Calculating feature importance...")
importance_gain = model.booster_.feature_importance(importance_type="gain")
importance_split = model.booster_.feature_importance(importance_type="split")
feat_names = np.array(feature_cols)

# 保留原始顺序
imp_df_original = pd.DataFrame(
    {
        "feature": feat_names,
        "importance_gain": importance_gain,
        "importance_split": importance_split,
    }
)
orig_path = os.path.join(OUTPUT_DIR, "feature_importance_original_order.csv")
safe_to_csv(imp_df_original, orig_path, index=False, encoding="utf-8-sig")
print(f"Saved original-order feature importance to {orig_path}")

# 按 Gain 降序排列
imp_df = imp_df_original.sort_values("importance_gain", ascending=False)
imp_path = os.path.join(OUTPUT_DIR, "feature_importance.csv")
safe_to_csv(imp_df, imp_path, index=False, encoding="utf-8-sig")
print(f"Saved sorted feature importance to {imp_path}")

# Top N 条形图
top_imp = imp_df.head(TOP_N)
plt.figure(figsize=(10, max(6, TOP_N * 0.25)))
plt.barh(top_imp["feature"][::-1], top_imp["importance_gain"][::-1])
plt.title(f"Top {TOP_N} Feature Importance (Gain)")
plt.xlabel("Importance Gain")
plt.tight_layout()
fig_path = os.path.join(OUTPUT_DIR, "feature_importance_top50.png")
safe_save_fig(fig_path)
plt.close()
print("Saved feature importance plot.")

# ====== 准备 SHAP 采样数据 ======
print("Computing SHAP values...")
sample_size = min(5000, X_train.shape[0])
sample_indices = np.random.choice(X_train.index, sample_size, replace=False)
X_sample = X_train.loc[sample_indices]

# ====== 辅助函数 ======
def mean_abs_shap(arr: np.ndarray) -> np.ndarray:
    """对每个特征计算 mean(|SHAP|)。输入必须为 (n_samples, n_features)"""
    return np.mean(np.abs(arr), axis=0)


def build_top_df(mean_abs: np.ndarray, features: np.ndarray, top_k: int = TOP_N, cls_name=None):
    """根据 mean(|SHAP|) 构建 Top 特征 DataFrame"""
    idx = np.argsort(-mean_abs)
    df_top = pd.DataFrame(
        {
            "feature": features[idx][:top_k],
            "mean_abs_shap": mean_abs[idx][:top_k],
        }
    )
    if cls_name is not None:
        df_top.insert(0, "class", cls_name)
    return df_top


def to_classwise_shap_list(shap_values_raw, n_samples, n_features, n_classes):
    """
    将 shap_values 统一转成：
        [array(shape=(n_samples, n_features)), ...]  长度 = n_classes

    支持：
    - list: [ (n_samples, n_features), ... ]
    - 3D ndarray: 维度可能是 (n_samples, n_classes, n_features)、
                  (n_samples, n_features, n_classes)、
                  (n_classes, n_samples, n_features) 等。
    """
    # 已经是按类别 list 的情况
    if isinstance(shap_values_raw, list):
        if len(shap_values_raw) != n_classes:
            raise ValueError(
                f"SHAP 返回 list，但长度 {len(shap_values_raw)} "
                f"与类别数 {n_classes} 不一致"
            )
        # 每个元素应为 (n_samples, n_features)
        for arr in shap_values_raw:
            if not isinstance(arr, np.ndarray) or arr.ndim != 2:
                raise ValueError("list 中每个 shap 数组必须是二维 ndarray")
        return shap_values_raw

    # ndarray 情况
    if not isinstance(shap_values_raw, np.ndarray):
        raise ValueError(f"无法处理的 shap_values 类型: {type(shap_values_raw)}")

    if shap_values_raw.ndim == 2:
        # 2D 的情况（有些版本二分类会这样），复制成每类一份
        if shap_values_raw.shape != (n_samples, n_features):
            raise ValueError(
                f"二维 shap_values 的形状 {shap_values_raw.shape} "
                f"与 (n_samples, n_features)=({n_samples}, {n_features}) 不一致"
            )
        return [shap_values_raw for _ in range(n_classes)]

    if shap_values_raw.ndim == 3:
        # 自动识别哪一维是 samples / classes / features
        s = shap_values_raw.shape
        try:
            ax_samples = [i for i in range(3) if s[i] == n_samples][0]
            ax_classes = [i for i in range(3) if s[i] == n_classes and i != ax_samples][0]
            ax_features = [
                i
                for i in range(3)
                if s[i] == n_features and i not in (ax_samples, ax_classes)
            ][0]
        except IndexError:
            raise ValueError(
                f"无法根据形状 {s} 匹配 (n_samples={n_samples}, "
                f"n_classes={n_classes}, n_features={n_features})"
            )

        # 重排为 (n_classes, n_samples, n_features)
        shap_reordered = np.moveaxis(
            shap_values_raw, [ax_classes, ax_samples, ax_features], [0, 1, 2]
        )
        # 按类别拆分，每个 (n_samples, n_features)
        return [shap_reordered[c, :, :] for c in range(n_classes)]

    raise ValueError(f"不支持的 shap_values 维度: {shap_values_raw.ndim}")


shap_top_df = None

# ====== 优先使用 SHAP，失败再兜底 ======
if shap_available:
    try:
        # 对树模型用 TreeExplainer 更快
        explainer = shap.TreeExplainer(model)
        shap_values_raw = explainer.shap_values(
            X_sample, check_additivity=False  # 防止部分版本出现 additivity warning/error
        )

        if is_multiclass:
            # 多分类：统一转成 [ (n_samples, n_features) ] 的列表
            n_samples, n_features = X_sample.shape
            n_classes = len(classes)
            shap_values_per_class = to_classwise_shap_list(
                shap_values_raw, n_samples, n_features, n_classes
            )

            # 画第一个类别的 summary plot（可按需改成循环）
            plt.figure()
            shap.summary_plot(shap_values_per_class[0], X_sample, show=False)
            plt.tight_layout()
            shap_fig_path = os.path.join(OUTPUT_DIR, "shap_summary.png")
            safe_save_fig(shap_fig_path)
            plt.close()

            # 计算每个类别的 Top 特征
            dfs = []
            for cls_idx, cls_name in enumerate(classes):
                mean_abs = mean_abs_shap(shap_values_per_class[cls_idx])
                dfs.append(build_top_df(mean_abs, feat_names, cls_name=cls_name))
            shap_top_df = pd.concat(dfs, ignore_index=True)

        else:
            # 二分类 / 回归：确保是 2D
            if isinstance(shap_values_raw, list):
                shap_values_2d = shap_values_raw[0]
            else:
                shap_values_2d = shap_values_raw

            if not isinstance(shap_values_2d, np.ndarray) or shap_values_2d.ndim != 2:
                raise ValueError(f"二分类 SHAP 形状异常: {type(shap_values_2d)}, ndim={shap_values_2d.ndim}")

            plt.figure()
            shap.summary_plot(shap_values_2d, X_sample, show=False)
            plt.tight_layout()
            shap_fig_path = os.path.join(OUTPUT_DIR, "shap_summary.png")
            safe_save_fig(shap_fig_path)
            plt.close()

            mean_abs = mean_abs_shap(shap_values_2d)
            shap_top_df = build_top_df(mean_abs, feat_names)

    except Exception as e:
        shap_available = False
        print(f"SHAP 计算出错，改用 LightGBM 兜底：{e}")

# ====== 如果 SHAP 不可用 / 失败，使用 pred_contrib 兜底 ======
if not shap_available:
    raw_contrib = model.booster_.predict(
        X_sample, pred_contrib=True, num_iteration=model.best_iteration_
    )
    n_samples = X_sample.shape[0]
    n_features = len(feat_names)
    n_classes = len(classes) if is_multiclass else 1

    if is_multiclass:
        # LightGBM multi-class pred_contrib 输出形状一般为 (n_samples, n_classes * (n_features + 1))
        raw_contrib = raw_contrib.reshape(n_samples, n_classes, n_features + 1)
        dfs = []
        for cls_idx, cls_name in enumerate(classes):
            contrib_cls = raw_contrib[:, cls_idx, :-1]  # 去掉 bias 项
            dfs.append(
                build_top_df(mean_abs_shap(contrib_cls), feat_names, cls_name=cls_name)
            )
        shap_top_df = pd.concat(dfs, ignore_index=True)

        # 画第一个类别的 TopN 作为可视化
        plt.figure(figsize=(10, max(6, TOP_N * 0.25)))
        subset = dfs[0].head(TOP_N)
        plt.barh(subset["feature"][::-1], subset["mean_abs_shap"][::-1])
        plt.title(f"Top {TOP_N} SHAP (pred_contrib fallback, class={classes[0]})")
        plt.xlabel("Mean |SHAP|")
        plt.tight_layout()
        shap_fig_path = os.path.join(OUTPUT_DIR, "shap_summary.png")
        safe_save_fig(shap_fig_path)
        plt.close()

    else:
        contrib = raw_contrib[:, :-1]  # 去掉 bias
        mean_abs = mean_abs_shap(contrib)
        shap_top_df = build_top_df(mean_abs, feat_names)

        plt.figure(figsize=(10, max(6, TOP_N * 0.25)))
        subset = shap_top_df.head(TOP_N)
        plt.barh(subset["feature"][::-1], subset["mean_abs_shap"][::-1])
        plt.title(f"Top {TOP_N} SHAP (pred_contrib fallback)")
        plt.xlabel("Mean |SHAP|")
        plt.tight_layout()
        shap_fig_path = os.path.join(OUTPUT_DIR, "shap_summary.png")
        safe_save_fig(shap_fig_path)
        plt.close()

# ====== 保存 SHAP Top 特征 ======
shap_top_path = os.path.join(OUTPUT_DIR, "shap_top_features.csv")
safe_to_csv(shap_top_df, shap_top_path, index=False, encoding="utf-8-sig")
print(f"Saved SHAP top features to {shap_top_path}")
print("【完成】所有结果保存在目录：", OUTPUT_DIR)

Loading data from: E:\科研\深度学习框架--推荐系统\user_item_purchase_matrix_full_latest_with_capacity.csv
UserID column: 'customerID'
Target column: 'investmentCapacity'
Number of features: 806
Unique classes in target: [0 1 2 3 4]
Training samples: 20944, Validation samples: 2328, Test samples: 5818
Training LightGBM model (quiet)...
Evaluating on test data...
              precision    recall  f1-score   support

           0       0.76      0.58      0.66      3543
           1       0.25      0.11      0.16      1097
           2       0.35      0.27      0.30       953
           3       0.07      0.36      0.12       157
           4       0.03      0.49      0.06        68

    accuracy                           0.43      5818
   macro avg       0.29      0.36      0.26      5818
weighted avg       0.57      0.43      0.48      5818

Calculating feature importance...
[警告] 无法写入 E:\科研\深度学习框架--推荐系统\lgbm_shap_outputs\feature_importance_original_order.csv（权限不足或被占用），改写入 E:\科研\深度学习框架--推荐系统\lgbm_sh